In [5]:
#Import the requisite library
#import spacy

In [2]:
# #Sample text
# text = "This is a sample number 555-5555."

# #Build upon the spaCy Small Model
# nlp = spacy.blank("en")

# #Create the Ruler and Add it
# ruler = nlp.add_pipe("entity_ruler")

# #List of Entities and Patterns (source: https://spacy.io/usage/rule-based-matching)
# patterns = [
#                 {"label": "PHONE_NUMBER", "pattern": [{"SHAPE": "ddd"},
#                 {"ORTH": "-", "OP": "?"}, {"SHAPE": "dddd"}]}
#             ]
# #add patterns to ruler
# ruler.add_patterns(patterns)

# #create the doc
# doc = nlp(text)

# #extract entities
# for ent in doc.ents:
#     print (ent.text, ent.label_)

555-5555 PHONE_NUMBER


In [3]:
# #Sample text
# text = "This is a sample number (555) 555-5555."

# #Build upon the spaCy Small Model
# nlp = spacy.blank("en")

# #Create the Ruler and Add it
# ruler = nlp.add_pipe("entity_ruler")

# #List of Entities and Patterns (source: https://spacy.io/usage/rule-based-matching)
# patterns = [
#                 {
#                     "label": "PHONE_NUMBER", "pattern": [{"TEXT": {"REGEX": "((\d){3}-(\d){4})"}}
#                                                         ]
#                 }
#             ]
# #add patterns to ruler
# ruler.add_patterns(patterns)


# #create the doc
# doc = nlp(text)

# #extract entities
# for ent in doc.ents:
#     print (ent.text, ent.label_)
# #"This is for one very important reason. SpaCy’s EntityRuler cannot use RegEx to pattern match across tokens.
# #  The dash in the phone number throws off the EntityRuler."

In [6]:
import re

In [7]:
text = "Paul Newman was an American actor, but Paul Hollywood is a British TV Host. The name Paul is quite common."

In [9]:
pattern = r"Paul [A-Z]\w+"
matches = re.finditer(pattern, text)
for match in matches:
    print(match)

<re.Match object; span=(0, 11), match='Paul Newman'>
<re.Match object; span=(39, 53), match='Paul Hollywood'>


In [10]:
import spacy
from spacy.tokens import Span

In [11]:
nlp = spacy.blank("en")
doc = nlp(text)

In [13]:
original_ents = list(doc.ents) #will be empty cause of no ner
mwt_ents = []
for match in re.finditer(pattern, text):
    start, end = match.span()
    span = doc.char_span(start, end)
    if span is not None:
        mwt_ents.append((span.start, span.end, span.text))

In [14]:
print(mwt_ents)

[(0, 2, 'Paul Newman'), (8, 10, 'Paul Hollywood')]


In [17]:
for ent in mwt_ents:
    start, end, name = ent
    per_ent = Span(doc, start, end, label = "PERSON")
    original_ents.append(per_ent)
    doc.ents = original_ents
for ent in doc.ents:
    print(ent.text, ent.label_)

Paul Newman PERSON
Paul Hollywood PERSON


In [25]:
#trying to put this into an actual custom pipeline
#THIS IS IMPLEMENTING REGEX ACROSS MULTIPLE TOKENSnl
from spacy.language import Language
@Language.component("paul_ner")
def paul_ner(doc):
    pattern = r"Paul [A-Z]\w+"
    original_ents = list(doc.ents) #will be empty cause of no ner
    mwt_ents = []
    for match in re.finditer(pattern, text):
        start, end = match.span()
        span = doc.char_span(start, end)
        if span is not None:
            mwt_ents.append((span.start, span.end, span.text))
    for ent in mwt_ents:
        start, end, name = ent
        per_ent = Span(doc, start, end, label = "PERSON")
        original_ents.append(per_ent)
    doc.ents = original_ents
    
    
    return(doc)

In [26]:
nlp2 = spacy.blank("en")
nlp2.add_pipe("paul_ner")

<function __main__.paul_ner(doc)>

In [28]:
doc2 = nlp2(text)
print(doc.ents)

(Paul Newman, Paul Hollywood)
